In [ ]:
# !pip install opencv-python-headless
# !pip install imageio
# !pip install OpenEXR
# !pip install Imath

import os
os.environ["OPENCV_IO_ENABLE_OPENEXR"]="1"

import os
print(os.environ.get('OPENCV_IO_ENABLE_OPENEXR'))

import cv2
import imageio
import torch
import torch.nn as nn
import torch.nn.functional as F
# !pip install torchinfo
from torchinfo import summary

import OpenEXR
import Imath
import numpy as np

import glob
from google.colab import drive
drive.mount('/content/drive/')

1
Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
!pip uninstall opencv-python-headless -y
!pip uninstall opencv-python -y
!apt update
!apt install python3-opencv

Found existing installation: opencv-python-headless 4.9.0.80
Uninstalling opencv-python-headless-4.9.0.80:
  Successfully uninstalled opencv-python-headless-4.9.0.80
Found existing installation: opencv-python 4.8.0.76
Uninstalling opencv-python-4.8.0.76:
  Successfully uninstalled opencv-python-4.8.0.76
Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Get:7 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,081 kB]
Hit:8 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Get:9 http://archive.ubuntu.com/ubuntu jammy-updates/restricted amd64 Packages 

In [ ]:
import cv2
print(cv2.__version__)
print(cv2.getBuildInformation())

os.environ["OPENCV_IO_ENABLE_OPENEXR"]="1"

4.8.0

General configuration for OpenCV 4.8.0 =====================================
  Version control:               4.8.0-dirty

  Extra modules:
    Location (extra):            /io/opencv_contrib/modules
    Version control (extra):     4.8.0

  Platform:
    Timestamp:                   2023-08-09T11:58:06Z
    Host:                        Linux 5.15.0-1042-azure x86_64
    CMake:                       3.27.1
    CMake generator:             Unix Makefiles
    CMake build tool:            /bin/gmake
    Configuration:               Release

  CPU/HW features:
    Baseline:                    SSE SSE2 SSE3
      requested:                 SSE3
    Dispatched code generation:  SSE4_1 SSE4_2 FP16 AVX AVX2 AVX512_SKX
      requested:                 SSE4_1 SSE4_2 AVX FP16 AVX2 AVX512_SKX
      SSE4_1 (16 files):         + SSSE3 SSE4_1
      SSE4_2 (1 files):          + SSSE3 SSE4_1 POPCNT SSE4_2
      FP16 (0 files):            + SSSE3 SSE4_1 POPCNT SSE4_2 FP16 AVX
      AVX (7 files):

In [ ]:
#-------------------- LOAD DATASET HERE ----------------------------------
# !ls '/content/drive/MyDrive/hdr_dataset/hdr'
hdr_file_pattern = '/content/drive/MyDrive/hdr_dataset/hdr/*.exr'
# MyDrive/DIP Project/HDR Dataset 1'
# Loading one image:
hdr_images = []

for hdr_file_path in glob.glob(hdr_file_pattern):

    # Open the OpenEXR file
    hdr_file = OpenEXR.InputFile(hdr_file_path)

    # Get the header information
    header = hdr_file.header()

    # Get the image dimensions from the header
    dw = header['dataWindow']
    width = dw.max.x - dw.min.x + 1
    height = dw.max.y - dw.min.y + 1

    # Read the image data
    # Assuming the image has RGB channels, change it according to your case
    channel_names = ['R', 'G', 'B']  # Change this if your image has different channels
    for c in channel_names:
        print(c)
        # print(hdr_file.channel(c))
        print(type(hdr_file.channel(c)))
        # print(hdr_file.channel(c).shape
    data = [np.frombuffer(hdr_file.channel(c), dtype=np.float32) for c in channel_names]
    for d in data:
        print(d.shape)
        d = d.reshape((height, width))
    image_data = np.stack(data, axis=-1)
    break
# Now you have the image data in NumPy array format
print("Image dimensions:", image_data.shape)

R
<class 'bytes'>
G
<class 'bytes'>
B
<class 'bytes'>
(1036800,)


ValueError: cannot reshape array of size 1036800 into shape (1080,1920)

In [ ]:
# -------------------- IGNORE THIS BLOCK AT THE MOMENT --------------------

def discriminator_loss(D, real_images, fake_images):
  real_loss = F.mse_loss(D(real_images), torch.ones_like(D(real_images)))
  fake_loss = F.mse_loss(D(fake_images), torch.zeros_like(D(fake_images)))
  return real_loss + fake_loss

def generator_loss(G, fake_images):
  return F.mse_loss(G(fake_images), torch.ones_like(G(fake_images)))

# Optional: L1 Content Loss to enforce similarity between generated and real LDR images
def l1_content_loss(fake_images, real_images):
    return torch.mean(torch.abs(fake_images - real_images))


# def detector_loss():
#   # TBD
#   pass

In [ ]:
# -------------------- BASIC GAN LOSSES -------------------------

def generator_adversarial_loss(D, fake_images):
    return torch.mean((D(fake_images) - 1) ** 2)

def discriminator_adverserial_loss(D, real_images, fake_images):
    real_loss = torch.mean((D(real_images) - 1) ** 2)
    fake_loss = torch.mean(D(fake_images) ** 2)
    return (real_loss + fake_loss) / 2

In [ ]:
# ------------------------- GENERATOR ARCHITECTURE -------------------------

class ConvBlock(nn.Module):
  def __init__(self, in_channels, out_channels, kernel_size=3, stride=1, padding=1):
    super(ConvBlock, self).__init__()
    self.conv = nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding)
    self.ins = nn.InstanceNorm2d(out_channels)
    self.relu = nn.ReLU()

  def forward(self, x):
      x = self.conv(x)
      x = self.ins(x)
      x = self.relu(x)
      return x

class AttentionModule(nn.Module):
  def __init__(self, in_channels, out_channels):
    super(AttentionModule, self).__init__()
    self.attention_score = nn.Conv2d(in_channels, in_channels, kernel_size=1)
    self.sigmoid = nn.Sigmoid()

  def forward(self, x):
      score = self.attention_score(x)
      score = self.sigmoid(score)
      attention_map = torch.mul(score, x)
      return attention_map

class Generator(nn.Module):
    def __init__(self, in_channels=3):
        super(Generator, self).__init__()
        self.conv1 = ConvBlock(in_channels, 32, kernel_size=1, stride=1, padding=0)
        self.conv2 = ConvBlock(32, 64, kernel_size=1, stride=1, padding=0)
        self.conv3 = ConvBlock(64, 128, kernel_size=1, stride=1, padding=0)
        self.conv4 = ConvBlock(512, 128, kernel_size=1, stride=1, padding=0)
        self.conv5 = ConvBlock(128, 64, kernel_size=1, stride=1, padding=0)
        self.conv6 = ConvBlock(64, 3, kernel_size=1, stride=1, padding=0)

        self.attention1 = AttentionModule(32, 32)
        self.attention2 = AttentionModule(64, 64)
        self.attention3 = AttentionModule(512, 512)

        self.k3 = nn.Conv2d(128, 128, kernel_size=3, stride=1, padding=1)
        self.k5 = nn.Conv2d(128, 128, kernel_size=5, stride=1, padding=2)
        self.k7 = nn.Conv2d(128, 128, kernel_size=7, stride=1, padding=3)
        self.k9 = nn.Conv2d(128, 128, kernel_size=9, stride=1, padding=4)

        self.maxpool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.upsample = nn.Upsample(scale_factor=2, mode='nearest') # TESTING

    def forward(self, x):
        x = self.conv1(x)
        x = self.maxpool(x)
        att_1 = self.attention1(x)
        x = self.conv2(x)
        x = self.maxpool(x)
        att_2 = self.attention2(x)
        x = self.conv3(x)
        x = self.maxpool(x)
        x_1 = self.k3(x)
        x_2 = self.k5(x)
        x_3 = self.k7(x)
        x_4 = self.k9(x)
        x = torch.cat((x_1, x_2, x_3, x_4), dim=1) # TESTING
        att_3 = self.attention3(x)
        x = self.upsample(att_3) # CONTINUE CODING HERE
        x = self.conv4(x)
        x = self.upsample(x)
        x = self.conv5(x)
        x = self.upsample(x)
        x = self.conv6(x)
        return x


In [ ]:
# ------------------------- DISCRIMINATOR ARCHITECTURE -------------------------

class Discriminator(nn.Module):
  def __init__(self, in_channels=3):
    super(Discriminator, self).__init__()

    self.model = nn.Sequential(
        # Block 1 Input -> Channels x H x W
        nn.Conv2d(in_channels, 64, kernel_size=4, stride=2, padding=1),
        nn.LeakyReLU(0.2, inplace=True),

        # Block 2 Input -> 64 * H/2 * W/2
        nn.Conv2d(64, 128, kernel_size=4, stride=2, padding=1),
        nn.InstanceNorm2d(128),
        nn.LeakyReLU(0.2, inplace=True),

        nn.Conv2d(128, 256, kernel_size=4, stride=2, padding=1),
        nn.InstanceNorm2d(256),
        nn.LeakyReLU(0.2, inplace=True),

        nn.Conv2d(256, 512, kernel_size=4, stride=1, padding=1),
        nn.InstanceNorm2d(512),
        nn.LeakyReLU(0.2, inplace=True),

        nn.Conv2d(512, 1, kernel_size=4, stride=1, padding=1)
    )

  def forward(self, x):
      return self.model(x)

In [ ]:
# --------------------- PATCH DISCRIMINATOR -------------------

class PatchDiscriminator(nn.Module):
    def __init__(self, in_channels=3):
        super(PatchDiscriminator, self).__init__()

        # Define layers
        self.conv1 = nn.Conv2d(in_channels, 64, kernel_size=4, stride=2, padding=1)
        self.conv2 = nn.Conv2d(64, 128, kernel_size=4, stride=2, padding=1)
        self.conv3 = nn.Conv2d(128, 256, kernel_size=4, stride=2, padding=1)
        self.conv4 = nn.Conv2d(256, 512, kernel_size=4, stride=1, padding=1)
        self.conv5 = nn.Conv2d(512, 1, kernel_size=4, stride=1, padding=1)

        # Instance normalization
        self.instance_norm = nn.InstanceNorm2d(512)

    def forward(self, x):
        # Forward pass through convolutional layers
        x = nn.LeakyReLU(0.2)(self.conv1(x))
        x = nn.LeakyReLU(0.2)(self.instance_norm(self.conv2(x)))
        x = nn.LeakyReLU(0.2)(self.instance_norm(self.conv3(x)))
        x = nn.LeakyReLU(0.2)(self.instance_norm(self.conv4(x)))
        x = self.conv5(x)

        return x

In [ ]:
H = 256
W = 256
batch_size = 1
in_channels =3
# Testing

gen_model = Generator(in_channels=3)
print(summary(gen_model, input_size=(batch_size, in_channels, H, W)))

Layer (type:depth-idx)                   Output Shape              Param #
Generator                                [1, 3, 256, 256]          --
├─ConvBlock: 1-1                         [1, 32, 256, 256]         --
│    └─Conv2d: 2-1                       [1, 32, 256, 256]         128
│    └─InstanceNorm2d: 2-2               [1, 32, 256, 256]         --
│    └─ReLU: 2-3                         [1, 32, 256, 256]         --
├─MaxPool2d: 1-2                         [1, 32, 128, 128]         --
├─AttentionModule: 1-3                   [1, 32, 128, 128]         --
│    └─Conv2d: 2-4                       [1, 32, 128, 128]         1,056
│    └─Sigmoid: 2-5                      [1, 32, 128, 128]         --
├─ConvBlock: 1-4                         [1, 64, 128, 128]         --
│    └─Conv2d: 2-6                       [1, 64, 128, 128]         2,112
│    └─InstanceNorm2d: 2-7               [1, 64, 128, 128]         --
│    └─ReLU: 2-8                         [1, 64, 128, 128]         --
├─MaxPoo

In [ ]:
patch_disc_model = PatchDiscriminator(in_channels=3)
print(summary(patch_disc_model, input_size=(batch_size, in_channels, H, W)))

Layer (type:depth-idx)                   Output Shape              Param #
PatchDiscriminator                       [1, 1, 30, 30]            --
├─Conv2d: 1-1                            [1, 64, 128, 128]         3,136
├─Conv2d: 1-2                            [1, 128, 64, 64]          131,200
├─InstanceNorm2d: 1-3                    [1, 128, 64, 64]          --
├─Conv2d: 1-4                            [1, 256, 32, 32]          524,544
├─InstanceNorm2d: 1-5                    [1, 256, 32, 32]          --
├─Conv2d: 1-6                            [1, 512, 31, 31]          2,097,664
├─InstanceNorm2d: 1-7                    [1, 512, 31, 31]          --
├─Conv2d: 1-8                            [1, 1, 30, 30]            8,193
Total params: 2,764,737
Trainable params: 2,764,737
Non-trainable params: 0
Total mult-adds (G): 3.15
Input size (MB): 0.79
Forward/backward pass size (MB): 18.62
Params size (MB): 11.06
Estimated Total Size (MB): 30.47


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/instancenorm.py:80: UserWarning: input's size at dim=1 does not match num_features. You can silence this warning by not passing in num_features, which is not used because affine=False
  warnings.warn(f"input's size at dim={feature_dim} does not match num_features. "


In [ ]:
disc_model = Discriminator(in_channels=3)
print(summary(disc_model, input_size=(batch_size, in_channels, H, W)))

Layer (type:depth-idx)                   Output Shape              Param #
Discriminator                            [1, 1, 30, 30]            --
├─Sequential: 1-1                        [1, 1, 30, 30]            --
│    └─Conv2d: 2-1                       [1, 64, 128, 128]         3,136
│    └─LeakyReLU: 2-2                    [1, 64, 128, 128]         --
│    └─Conv2d: 2-3                       [1, 128, 64, 64]          131,200
│    └─InstanceNorm2d: 2-4               [1, 128, 64, 64]          --
│    └─LeakyReLU: 2-5                    [1, 128, 64, 64]          --
│    └─Conv2d: 2-6                       [1, 256, 32, 32]          524,544
│    └─InstanceNorm2d: 2-7               [1, 256, 32, 32]          --
│    └─LeakyReLU: 2-8                    [1, 256, 32, 32]          --
│    └─Conv2d: 2-9                       [1, 512, 31, 31]          2,097,664
│    └─InstanceNorm2d: 2-10              [1, 512, 31, 31]          --
│    └─LeakyReLU: 2-11                   [1, 512, 31, 31]        

In [ ]:
# ------------------ TRAINING -------------------

# Instantiate Generator and Discriminator
generator = Generator()
discriminator = Discriminator()

# Define loss function and optimizers
criterion = nn.BCEWithLogitsLoss()
generator_optimizer = torch.optim.Adam(generator.parameters(), lr=0.0002, betas=(0.5, 0.999))
discriminator_optimizer = torch.optim.Adam(discriminator.parameters(), lr=0.0002, betas=(0.5, 0.999))

num_epochs = 100
for epoch in range(num_epochs):
    for batch_idx, (real_images, _) in enumerate(train_loader):
        batch_size = real_images.size(0)

        # Train Discriminator
        discriminator_optimizer.zero_grad()

        # Train with real images
        real_labels = torch.ones(batch_size, 1)
        output = discriminator(real_images)
        real_loss = criterion(output, real_labels)

        # Train with fake images
        fake_labels = torch.zeros(batch_size, 1)
        # noise = torch.randn(batch_size, input_dim, 1, 1)
        fake_images = generator(hdr_images) # NEED HDR IMAGES
        output = discriminator(fake_images.detach())
        fake_loss = criterion(output, fake_labels)

        discriminator_loss = real_loss + fake_loss
        discriminator_loss.backward()
        discriminator_optimizer.step()

        # Train Generator
        generator_optimizer.zero_grad()

        # noise = torch.randn(batch_size, noise_dim, 1, 1)
        fake_images = generator(hdr_images) # NEED HDR IMAGES
        output = discriminator(fake_images)
        generator_loss = criterion(output, real_labels)

        generator_loss.backward()
        generator_optimizer.step()

        # Print training losses
        if batch_idx % 100 == 0:
            print(f"Epoch [{epoch}/{num_epochs}], Batch [{batch_idx}/{len(train_loader)}], "
                  f"Discriminator Loss: {discriminator_loss.item():.4f}, "
                  f"Generator Loss: {generator_loss.item():.4f}")
